# CREACION DE LOS DATASET PARA LOS DASHBOARD

## IMPORTS GLOBALES

In [58]:
import pandas as pd
import numpy as np
import  requests
import io
from matplotlib import pyplot as plt
import seaborn as sns

## LECTURA DE ARCHIVOS

In [60]:
df_bandas = pd.read_csv('data/bandas_provinciales.csv')
df_servicios = pd.read_csv('data/servicios_provinciales.csv')
df_porcentajes = pd.read_csv('data/porcentaje_provincias.csv')

In [61]:
df_dinero = pd.read_csv('data/dinero-trimestral.csv')
df_velocidad = pd.read_csv('data/velocidad.csv')
df_no_prov = pd.read_csv('data/bandas_no_prov.csv')

## TRATAMIENTO A LOS CSV

### BANDAS CSV

In [62]:
df_bandas.head(10)

,Año,Trimestre,Provincia,Banda ancha fija,Dial up
0,2022,4,Buenos Aires,5,6.0
1,2022,4,Capital Federal,2,2.0
2,2022,4,Catamarca,71,1.0
3,2022,4,Chaco,144,5.0
4,2022,4,Chubut,168,904.0
5,2022,4,Córdoba,1,68.0
6,2022,4,Corrientes,155,4.0
7,2022,4,Entre Ríos,271,22.0
8,2022,4,Formosa,62,0.0
9,2022,4,Jujuy,118,0.0


In [63]:
df_bandas['Provincia'] = df_bandas['Provincia'].str.upper()
df_bandas

,Año,Trimestre,Provincia,Banda ancha fija,Dial up
0,2022,4,BUENOS AIRES,5,6.0
1,2022,4,CAPITAL FEDERAL,2,2.0
2,2022,4,CATAMARCA,71,1.0
3,2022,4,CHACO,144,5.0
4,2022,4,CHUBUT,168,904.0
...,...,...,...,...,...
859,2014,1,SANTA CRUZ,26,460.0
860,2014,1,SANTA FE,506,612.0
861,2014,1,SANTIAGO DEL ESTERO,37,9.0
862,2014,1,TIERRA DEL FUEGO,27,766.0


### PORCENTAJES CSV

In [64]:
df_porcentajes

,Año,Trimestre,Provincia,accesos_%home_prov
0,2022,4,Buenos Aires,78.94
1,2022,4,Capital Federal,122.73
2,2022,4,Catamarca,65.77
3,2022,4,Chaco,43.62
4,2022,4,Chubut,85.57
...,...,...,...,...
859,2014,1,Santa Cruz,28.94
860,2014,1,Santa Fe,47.85
861,2014,1,Santiago Del Estero,16.31
862,2014,1,Tierra Del Fuego,63.97


In [65]:
df_porcentajes['Provincia'] = df_porcentajes['Provincia'].str.upper()
df_porcentajes

,Año,Trimestre,Provincia,accesos_%home_prov
0,2022,4,BUENOS AIRES,78.94
1,2022,4,CAPITAL FEDERAL,122.73
2,2022,4,CATAMARCA,65.77
3,2022,4,CHACO,43.62
4,2022,4,CHUBUT,85.57
...,...,...,...,...
859,2014,1,SANTA CRUZ,28.94
860,2014,1,SANTA FE,47.85
861,2014,1,SANTIAGO DEL ESTERO,16.31
862,2014,1,TIERRA DEL FUEGO,63.97


In [66]:
def get_trimestre_abs(trimestre,anio):
  return trimestre+4*(anio-df_porcentajes["Año"].min())

df_porcentajes["TrimestreAbs"] = df_porcentajes.apply(lambda r:get_trimestre_abs(r["Trimestre"],r["Año"]),axis=1)
df_porcentajes

,Año,Trimestre,Provincia,accesos_%home_prov,TrimestreAbs
0,2022,4,BUENOS AIRES,78.94,36
1,2022,4,CAPITAL FEDERAL,122.73,36
2,2022,4,CATAMARCA,65.77,36
3,2022,4,CHACO,43.62,36
4,2022,4,CHUBUT,85.57,36
...,...,...,...,...,...
859,2014,1,SANTA CRUZ,28.94,1
860,2014,1,SANTA FE,47.85,1
861,2014,1,SANTIAGO DEL ESTERO,16.31,1
862,2014,1,TIERRA DEL FUEGO,63.97,1


In [67]:
df_porcentajes[df_porcentajes["Año"]==2015]

,Año,Trimestre,Provincia,accesos_%home_prov,TrimestreAbs
672,2015,4,BUENOS AIRES,58.29,8
673,2015,4,CAPITAL FEDERAL,118.13,8
674,2015,4,CATAMARCA,26.24,8
675,2015,4,CHACO,28.35,8
676,2015,4,CHUBUT,36.89,8
...,...,...,...,...,...
763,2015,1,SANTA CRUZ,27.99,5
764,2015,1,SANTA FE,50.24,5
765,2015,1,SANTIAGO DEL ESTERO,17.06,5
766,2015,1,TIERRA DEL FUEGO,74.23,5


### SERVICIOS CSV

In [68]:
df_servicios.head()

,Provincia,ADSL,Cablemódem,Dial Up,Fibra óptica,4G,3G,Telefonía Fija,Wireless,Satelital
0,BUENOS AIRES,True,False,False,False,True,True,True,False,False
1,BUENOS AIRES,False,False,False,False,True,True,True,False,False
2,BUENOS AIRES,False,False,True,False,True,True,True,True,False
3,BUENOS AIRES,False,False,False,False,True,False,False,False,False
4,BUENOS AIRES,True,False,False,True,True,True,True,False,False


In [69]:
def contar_servicio_y_agregar_columna(df, servicio, nombre_columna):
    counts = df[df[servicio] == True].groupby("Provincia")[servicio].sum()
    df[nombre_columna] = df["Provincia"].map(counts)

contar_servicio_y_agregar_columna(df_servicios, "ADSL", "ADSL_TOTAL")
contar_servicio_y_agregar_columna(df_servicios, "Cablemódem", "Cablemódem_TOTAL")
contar_servicio_y_agregar_columna(df_servicios, "Dial Up", "Dial_Up_TOTAL")
contar_servicio_y_agregar_columna(df_servicios, "Fibra óptica", "Fibra_óptica_TOTAL")
contar_servicio_y_agregar_columna(df_servicios, "4G", "4G_TOTAL")
contar_servicio_y_agregar_columna(df_servicios, "3G", "3G_TOTAL")
contar_servicio_y_agregar_columna(df_servicios, "Telefonía Fija", "Telefonía_Fija_TOTAL")
contar_servicio_y_agregar_columna(df_servicios, "Wireless", "Wireless_TOTAL")
contar_servicio_y_agregar_columna(df_servicios, "Satelital", "Satelital_TOTAL")


In [70]:
df_servicios.head(10)

,Provincia,ADSL,Cablemódem,Dial Up,Fibra óptica,4G,3G,Telefonía Fija,Wireless,Satelital,ADSL_TOTAL,Cablemódem_TOTAL,Dial_Up_TOTAL,Fibra_óptica_TOTAL,4G_TOTAL,3G_TOTAL,Telefonía_Fija_TOTAL,Wireless_TOTAL,Satelital_TOTAL
0,BUENOS AIRES,True,False,False,False,True,True,True,False,False,356,342,213.0,463,655,590,551,502,155
1,BUENOS AIRES,False,False,False,False,True,True,True,False,False,356,342,213.0,463,655,590,551,502,155
2,BUENOS AIRES,False,False,True,False,True,True,True,True,False,356,342,213.0,463,655,590,551,502,155
3,BUENOS AIRES,False,False,False,False,True,False,False,False,False,356,342,213.0,463,655,590,551,502,155
4,BUENOS AIRES,True,False,False,True,True,True,True,False,False,356,342,213.0,463,655,590,551,502,155
5,BUENOS AIRES,False,False,False,False,True,False,True,False,False,356,342,213.0,463,655,590,551,502,155
6,BUENOS AIRES,False,False,False,False,False,False,True,True,False,356,342,213.0,463,655,590,551,502,155
7,BUENOS AIRES,False,False,False,False,False,False,True,False,False,356,342,213.0,463,655,590,551,502,155
8,BUENOS AIRES,True,False,False,False,False,True,True,True,False,356,342,213.0,463,655,590,551,502,155
9,BUENOS AIRES,False,False,False,True,True,True,True,True,False,356,342,213.0,463,655,590,551,502,155


In [71]:
columnas_a_eliminar = ['ADSL', 'Cablemódem', 'Dial Up','Fibra óptica','4G','3G','Telefonía Fija','Wireless','Satelital']  # Lista de nombres de columnas a eliminar
df_servicios = df_servicios.drop(columns=columnas_a_eliminar)
df_servicios.head(10)

,Provincia,ADSL_TOTAL,Cablemódem_TOTAL,Dial_Up_TOTAL,Fibra_óptica_TOTAL,4G_TOTAL,3G_TOTAL,Telefonía_Fija_TOTAL,Wireless_TOTAL,Satelital_TOTAL
0,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155
1,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155
2,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155
3,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155
4,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155
5,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155
6,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155
7,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155
8,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155
9,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155


In [72]:
def calcular_total(fila):
    columnas_totales = ["ADSL_TOTAL",
    "Cablemódem_TOTAL",
    "Dial_Up_TOTAL",
    "Fibra_óptica_TOTAL",
    "4G_TOTAL",
    "3G_TOTAL",
    "Telefonía_Fija_TOTAL",
    "Wireless_TOTAL",
    "Satelital_TOTAL"]

    total = 0

    for columna in columnas_totales:
        total+=fila[columna]

    return total

df_servicios["Tecnologias_Total"] = df_servicios.apply(calcular_total,axis=1)
df_servicios

,Provincia,ADSL_TOTAL,Cablemódem_TOTAL,Dial_Up_TOTAL,Fibra_óptica_TOTAL,4G_TOTAL,3G_TOTAL,Telefonía_Fija_TOTAL,Wireless_TOTAL,Satelital_TOTAL,Tecnologias_Total
0,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155,3827.0
1,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155,3827.0
2,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155,3827.0
3,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155,3827.0
4,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155,3827.0
...,...,...,...,...,...,...,...,...,...,...,...
4306,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155,3827.0
4307,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155,3827.0
4308,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155,3827.0
4309,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155,3827.0


In [73]:
df_servicios = df_servicios.drop_duplicates()
df_servicios

,Provincia,ADSL_TOTAL,Cablemódem_TOTAL,Dial_Up_TOTAL,Fibra_óptica_TOTAL,4G_TOTAL,3G_TOTAL,Telefonía_Fija_TOTAL,Wireless_TOTAL,Satelital_TOTAL,Tecnologias_Total
0,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155,3827.0
44,MENDOZA,30,18,28.0,76,128,118,76,107,16,597.0
45,CABA,48,48,48.0,48,48,48,48,48,48,432.0
46,CATAMARCA,7,1,1.0,23,57,29,56,34,47,255.0
76,CORDOBA,179,88,5.0,175,344,277,318,291,73,1750.0
88,ENTRE RIOS,47,39,4.0,48,128,78,130,126,20,620.0
138,SALTA,32,16,1.0,86,88,43,64,71,66,467.0
168,SANTA CRUZ,5,5,7.0,9,16,15,17,10,22,106.0
180,SANTA FE,174,138,3.0,140,296,179,291,253,65,1539.0
370,CHACO,28,12,1.0,45,75,44,47,46,42,340.0


In [74]:
df_servicios_final = df_servicios.copy()

In [75]:
df_servicios_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 3112
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Provincia             24 non-null     object 
 1   ADSL_TOTAL            24 non-null     int64  
 2   Cablemódem_TOTAL      24 non-null     int64  
 3   Dial_Up_TOTAL         21 non-null     float64
 4   Fibra_óptica_TOTAL    24 non-null     int64  
 5   4G_TOTAL              24 non-null     int64  
 6   3G_TOTAL              24 non-null     int64  
 7   Telefonía_Fija_TOTAL  24 non-null     int64  
 8   Wireless_TOTAL        24 non-null     int64  
 9   Satelital_TOTAL       24 non-null     int64  
 10  Tecnologias_Total     21 non-null     float64
dtypes: float64(2), int64(8), object(1)
memory usage: 2.2+ KB


### DINERO TRIMESTRAL CSV

In [76]:
df_dinero.head(10)

,Año,Trimestre,Ingresos,Periodo
0,2013,1,13135235,Ene-Mar 2013
1,2013,2,13970806,Abr-Jun 2013
2,2013,3,15235659,Jul-Sept 2013
3,2013,4,16218846,Oct-Dic 2013
4,2014,1,15938353,Ene-Mar 2013
5,2014,2,17680972,Abr-Jun 2014
6,2014,3,18976053,Jul-Sept 2014
7,2014,4,20012332,Oct-Dic 2014
8,2015,1,19337342,Ene-Mar 2015
9,2015,2,19945453,Abr-Jun 2015


In [77]:
df_dinero=df_dinero.drop("Periodo", axis=1)
df_dinero.head(10)

,Año,Trimestre,Ingresos
0,2013,1,13135235
1,2013,2,13970806
2,2013,3,15235659
3,2013,4,16218846
4,2014,1,15938353
5,2014,2,17680972
6,2014,3,18976053
7,2014,4,20012332
8,2015,1,19337342
9,2015,2,19945453


### VELOCIDAD CSV

In [78]:
df_velocidad.head(10)

,Año,Trimestre,Provincia,512kbps,512kbps-1mbps,1mbps-6mbps,6mbps-10mbps,10mbps-20mbps,20mbps-30mbps,30mbps,Mbps
0,2022,4,Buenos Aires,28.934,23.347,259.866,281.550,244.555,98.670,3.760.109,111.000
1,2022,4,Capital Federal,516.000,5.096,31.320,64.653,36.336,8.116,1.299.117,18.286
2,2022,4,Catamarca,71.000,315.000,2.716,5.028,4.638,3.122,52.739,8.333
3,2022,4,Chaco,461.000,771.000,16.067,17.846,6.704,3.039,94.866,10.402
4,2022,4,Chubut,109.000,1.614,46.457,31.007,34.728,15.770,19.048,1.592
5,2022,4,Córdoba,99.000,9.929,143.686,108.195,63.120,20.691,686.330,10.877
6,2022,4,Corrientes,67.000,3.282,22.647,24.858,7.025,3.770,85.806,9.472
7,2022,4,Entre Ríos,107.000,4.722,45.629,46.700,20.927,22.483,130.406,8.747
8,2022,4,Formosa,84.000,245.000,22.458,12.391,6.724,310.000,18.930,7.203
9,2022,4,Jujuy,58.000,1.763,18.652,14.070,36.430,335.000,46.910,3.932


In [79]:
df_velocidad["Provincia"]= df_velocidad["Provincia"].str.upper()
df_velocidad.head(10)

,Año,Trimestre,Provincia,512kbps,512kbps-1mbps,1mbps-6mbps,6mbps-10mbps,10mbps-20mbps,20mbps-30mbps,30mbps,Mbps
0,2022,4,BUENOS AIRES,28.934,23.347,259.866,281.550,244.555,98.670,3.760.109,111.000
1,2022,4,CAPITAL FEDERAL,516.000,5.096,31.320,64.653,36.336,8.116,1.299.117,18.286
2,2022,4,CATAMARCA,71.000,315.000,2.716,5.028,4.638,3.122,52.739,8.333
3,2022,4,CHACO,461.000,771.000,16.067,17.846,6.704,3.039,94.866,10.402
4,2022,4,CHUBUT,109.000,1.614,46.457,31.007,34.728,15.770,19.048,1.592
5,2022,4,CÓRDOBA,99.000,9.929,143.686,108.195,63.120,20.691,686.330,10.877
6,2022,4,CORRIENTES,67.000,3.282,22.647,24.858,7.025,3.770,85.806,9.472
7,2022,4,ENTRE RÍOS,107.000,4.722,45.629,46.700,20.927,22.483,130.406,8.747
8,2022,4,FORMOSA,84.000,245.000,22.458,12.391,6.724,310.000,18.930,7.203
9,2022,4,JUJUY,58.000,1.763,18.652,14.070,36.430,335.000,46.910,3.932


### ACCESOS POR TECNOLOGIAS CSV

In [80]:
df_no_prov.head(10)

,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless
0,2022,4,1,6,3,573
1,2022,3,1,6,3,557
2,2022,2,1,6,3,556
3,2022,1,2,6,2,546
4,2021,4,2,6,2,523
5,2021,3,2,6,2,492
6,2021,2,2,6,1,477
7,2021,1,2,5,1,435
8,2020,4,2,5,1,422
9,2020,3,2,5,1,413


# DATASET FINALES

In [81]:
df_result = pd.merge(df_porcentajes, df_bandas, on=['Provincia', 'Año','Trimestre'])
df_result.head(10)

,Año,Trimestre,Provincia,accesos_%home_prov,TrimestreAbs,Banda ancha fija,Dial up
0,2022,4,BUENOS AIRES,78.94,36,5,6.0
1,2022,4,CAPITAL FEDERAL,122.73,36,2,2.0
2,2022,4,CATAMARCA,65.77,36,71,1.0
3,2022,4,CHACO,43.62,36,144,5.0
4,2022,4,CHUBUT,85.57,36,168,904.0
5,2022,4,CÓRDOBA,89.03,36,1,68.0
6,2022,4,CORRIENTES,51.64,36,155,4.0
7,2022,4,ENTRE RÍOS,64.01,36,271,22.0
8,2022,4,FORMOSA,39.41,36,62,0.0
9,2022,4,JUJUY,58.46,36,118,0.0


In [82]:
df_servicios.head(10)

,Provincia,ADSL_TOTAL,Cablemódem_TOTAL,Dial_Up_TOTAL,Fibra_óptica_TOTAL,4G_TOTAL,3G_TOTAL,Telefonía_Fija_TOTAL,Wireless_TOTAL,Satelital_TOTAL,Tecnologias_Total
0,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155,3827.0
44,MENDOZA,30,18,28.0,76,128,118,76,107,16,597.0
45,CABA,48,48,48.0,48,48,48,48,48,48,432.0
46,CATAMARCA,7,1,1.0,23,57,29,56,34,47,255.0
76,CORDOBA,179,88,5.0,175,344,277,318,291,73,1750.0
88,ENTRE RIOS,47,39,4.0,48,128,78,130,126,20,620.0
138,SALTA,32,16,1.0,86,88,43,64,71,66,467.0
168,SANTA CRUZ,5,5,7.0,9,16,15,17,10,22,106.0
180,SANTA FE,174,138,3.0,140,296,179,291,253,65,1539.0
370,CHACO,28,12,1.0,45,75,44,47,46,42,340.0


In [83]:
df_dinero_x_tec = pd.merge(df_no_prov,df_dinero, on=['Año','Trimestre'])
df_dinero_x_tec.head(10)

,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless,Ingresos
0,2022,4,1,6,3,573,204796575
1,2022,3,1,6,3,557,180356862
2,2022,2,1,6,3,556,153032313
3,2022,1,2,6,2,546,130454676
4,2021,4,2,6,2,523,127389786
5,2021,3,2,6,2,492,117751925
6,2021,2,2,6,1,477,104782020
7,2021,1,2,5,1,435,93112153
8,2020,4,2,5,1,422,83479114
9,2020,3,2,5,1,413,79634755


In [84]:
df_result.head(10)

,Año,Trimestre,Provincia,accesos_%home_prov,TrimestreAbs,Banda ancha fija,Dial up
0,2022,4,BUENOS AIRES,78.94,36,5,6.0
1,2022,4,CAPITAL FEDERAL,122.73,36,2,2.0
2,2022,4,CATAMARCA,65.77,36,71,1.0
3,2022,4,CHACO,43.62,36,144,5.0
4,2022,4,CHUBUT,85.57,36,168,904.0
5,2022,4,CÓRDOBA,89.03,36,1,68.0
6,2022,4,CORRIENTES,51.64,36,155,4.0
7,2022,4,ENTRE RÍOS,64.01,36,271,22.0
8,2022,4,FORMOSA,39.41,36,62,0.0
9,2022,4,JUJUY,58.46,36,118,0.0


In [85]:
df_servicios

,Provincia,ADSL_TOTAL,Cablemódem_TOTAL,Dial_Up_TOTAL,Fibra_óptica_TOTAL,4G_TOTAL,3G_TOTAL,Telefonía_Fija_TOTAL,Wireless_TOTAL,Satelital_TOTAL,Tecnologias_Total
0,BUENOS AIRES,356,342,213.0,463,655,590,551,502,155,3827.0
44,MENDOZA,30,18,28.0,76,128,118,76,107,16,597.0
45,CABA,48,48,48.0,48,48,48,48,48,48,432.0
46,CATAMARCA,7,1,1.0,23,57,29,56,34,47,255.0
76,CORDOBA,179,88,5.0,175,344,277,318,291,73,1750.0
88,ENTRE RIOS,47,39,4.0,48,128,78,130,126,20,620.0
138,SALTA,32,16,1.0,86,88,43,64,71,66,467.0
168,SANTA CRUZ,5,5,7.0,9,16,15,17,10,22,106.0
180,SANTA FE,174,138,3.0,140,296,179,291,253,65,1539.0
370,CHACO,28,12,1.0,45,75,44,47,46,42,340.0


In [86]:
df_servicios.isnull().sum()

Provincia               0
ADSL_TOTAL              0
Cablemódem_TOTAL        0
Dial_Up_TOTAL           3
Fibra_óptica_TOTAL      0
4G_TOTAL                0
3G_TOTAL                0
Telefonía_Fija_TOTAL    0
Wireless_TOTAL          0
Satelital_TOTAL         0
Tecnologias_Total       3
dtype: int64

In [87]:
df_servicios[df_servicios["Dial_Up_TOTAL"].isnull()]

,Provincia,ADSL_TOTAL,Cablemódem_TOTAL,Dial_Up_TOTAL,Fibra_óptica_TOTAL,4G_TOTAL,3G_TOTAL,Telefonía_Fija_TOTAL,Wireless_TOTAL,Satelital_TOTAL,Tecnologias_Total
422,FORMOSA,13,3,NaN,16,42,16,31,59,22,NaN
476,MISIONES,29,25,NaN,39,86,40,59,52,26,NaN
789,JUJUY,19,15,NaN,47,45,28,57,39,44,NaN


In [88]:
df_servicios["Dial_Up_TOTAL"]= df_servicios["Dial_Up_TOTAL"].replace(np.nan,0)

<ipython-input-88-a25d406ea8af>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_servicios["Dial_Up_TOTAL"]= df_servicios["Dial_Up_TOTAL"].replace(np.nan,0)


In [89]:
df_provincia_tec= df_servicios.copy() 

## EXPORTADO DE DATASETS

In [90]:
df_dinero_x_tec.head(10)

,Año,Trimestre,ADSL,Cablemodem,Fibra óptica,Wireless,Ingresos
0,2022,4,1,6,3,573,204796575
1,2022,3,1,6,3,557,180356862
2,2022,2,1,6,3,556,153032313
3,2022,1,2,6,2,546,130454676
4,2021,4,2,6,2,523,127389786
5,2021,3,2,6,2,492,117751925
6,2021,2,2,6,1,477,104782020
7,2021,1,2,5,1,435,93112153
8,2020,4,2,5,1,422,83479114
9,2020,3,2,5,1,413,79634755


In [92]:
df_provincia_tec.to_csv('data/provincias_tecnologias.csv',index=False)

In [93]:
df_din_tec.to_csv('data/din_tecnologias.csv', index=False)

In [94]:
df_velocidad.to_csv('data/provincia_velocidad.csv',index=False)

In [95]:
df_result.to_csv('data/porcentaaje.csv',index=False)